In [36]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
from pyspark.sql import functions as F

import time
start = time.time()

sc.stop()

sc = SparkContext()
sqlContext = SQLContext(sc)

rdd = sc.textFile("temperature-readings-tiny.csv")
parts = rdd.map(lambda l: l.split(";"))
df = parts.map(lambda p: Row(year=p[1].split("-")[0], value=float(p[3]),station=int(p[0]) ))

df = sqlContext.createDataFrame(df)	
df.registerTempTable("tempReadings")

df1 = df.filter(df.year.between(1950,2014))
df2 = df.groupBy("year",).agg({"value":"max"}).withColumnRenamed("max(value)","value")
df2 = df2.join(df1, ["year","value"],"inner").select("year", "station", "value")
df2 = df2.dropDuplicates(["year"])
df2 = df2.sort(df2.value.desc())

df3 = df.groupBy("year",).agg({"value":"min"}).withColumnRenamed("min(value)","value")
df3 = df3.join(df1, ["year","value"],"inner").select("year", "station", "value")
df3 = df3.dropDuplicates(["year"])
df3 = df3.sort(df3.value.desc())

# max_temperatureSorted = df2.rdd
# min_temperatureSorted = df3.rdd

# max_temperatureSorted.saveAsTextFile("./results2/2q1/max_temperatureSorted")
# min_temperatureSorted.saveAsTextFile("./results2/2q1/min_temperatureSorted")

end = time.time()
final_time = time.strftime('%H:%M:%S', time.gmtime(end-start))

print ("RUNTIME = " + final_time)


RUNTIME = 00:00:02
+----+-------+-----+
|year|station|value|
+----+-------+-----+
|2013| 102170|-13.3|
|1957| 102190|-19.9|
|1959| 102190|-23.2|
|1961| 102190|-23.5|
|2014| 102170|-24.3|
|1955| 102190|-26.2|
|1958| 102190|-27.9|
|1960| 102190|-28.3|
|1956| 102190|-30.0|
+----+-------+-----+

None
+----+-------+-----+
|year|station|value|
+----+-------+-----+
|2013| 102170|-13.3|
|1957| 102190|-19.9|
|1959| 102190|-23.2|
|1961| 102190|-23.5|
|2014| 102170|-24.3|
|1955| 102190|-26.2|
|1958| 102190|-27.9|
|1960| 102190|-28.3|
|1956| 102190|-30.0|
+----+-------+-----+

None
